# Generate “Passive Knowledge” Drills

That is, given a form, pitk the correct parsing (e.g. “fem. gen. pl.”).

## Imports

In [1]:

import almond.display.UpdatableDisplay
import almond.interpreter.api.DisplayData.ContentType
import almond.interpreter.api.{DisplayData, OutputHandler}

import java.io.File
import java.io.PrintWriter

import scala.io.Source
import scala.util.Random


import almond.display.UpdatableDisplay

import almond.interpreter.api.DisplayData.ContentType

import almond.interpreter.api.{DisplayData, OutputHandler}


import java.io.File

import java.io.PrintWriter


import scala.io.Source

import scala.util.Random


## Useful Functions

Load a file

In [2]:
def loadFile( fp: String ): Vector[String] = {
    Source.fromFile(fp).getLines.toVector
}


defined function loadFile

Save a string to a names file:

In [3]:
def saveString(s:String, filePath:String = "", fileName:String = "temp.txt"):Unit = {
		 val writer = new PrintWriter(new File(s"${filePath}${fileName}"))
         writer.write(s)
         writer.close()
	}

defined function saveString

Like `.split`, but preserving the character we split on:

In [4]:
def splitWithSplitter(text: String, puncs: String): Vector[String] = {
	//val regexWithSplitter = s"((?<=${puncs})|(?=${puncs}))"
    val regexWithSplitter = s"((?<=${puncs}))"
	text.split(regexWithSplitter).toVector.filter(_.size > 0)
}

defined function splitWithSplitter

Pretty Print Things:

In [5]:
def showMe(v:Any):Unit = {
  v match {
    case _:Vector[Any] => println(s"""\n----\n${v.asInstanceOf[Vector[Any]].mkString("\n")}\n----\n""")
    case _:Iterable[Any] => println(s"""\n----\n${v.asInstanceOf[Iterable[Any]].mkString("\n")}\n----\n""")
    case _ => println(s"\n-----\n${v}\n----\n")
  }
}

defined function showMe

Shuffle a Vector of Strings

In [6]:
def shuffle(v: Vector[String]): Vector[String] = {
    val rr: Vector[Float] = v.map( i => {
        scala.util.Random.nextFloat
    })
    val zipped: Vector[(String,Float)] = v.zip(rr)
    zipped.sortBy( m => m._2).map(_._1)
}

defined function shuffle

## GIFT-Generation

We load two files, a template and some morphological data. These get zipped together. This saves a lot of typing, since *all* "adjectives of three terminations" are going to have the same list of identifications ("m/n/s, f/n/s, …"), etc. 

We define our inputs (at the bottom), specify how many items we want in our output, and the name of the output file.

Finally, we invoke the command `makeActiveParadigmQuiz(…)`, which generates a set of multiple choice questions, in `.gift` format, ready for Moodle. 

For the record… when setting up a Moodle “quiz” that you mean to be a drill , select “Adaptive Mode (no penalties)” under “Question Behavor”. This lets your students click an answer, click “check”, see if it is right, and amend their answer if it was wrong. So they should *never* get to the end of a drill with a score lower than 100%.

What follows, below, are some functions that do this work. But we start with some basic Classes: `ParsedForm`,`IndexedParsedForm`, and `GradedParsedForm`.

## Classes

In [7]:
case class ParsedForm( parse: String, form: String)

defined class ParsedForm

In [8]:
case class IndexedParsedForm( index: Int, pform: ParsedForm)

defined class IndexedParsedForm

In [9]:
case class GradedForm( grade: Int, form: String)

defined class GradedForm

## Pick a Card

This function is for selecting the *wrong* answers for the multiple-choice drill. We have a *correct* answer, but we need a certain number of *wrong* answers.

The challenge is to avoid picking "bad" wrong answers. For example, we don't want, through random selection, to pick multiple instances of the original *correct* answer.

It is based on the metaphor of a deck of cards, from which we pick a certain number. The "deck" is a list of `IndexedParsedForm`s. We parameterize several things:

- How many to pick
- An initial one _not_ to pick

This function uses tail-recursion.

    (Before we do this, though, let's make a method to re-index a Vector[IndexedParsedForm]`.

In [10]:
def reindexParsedForms( idff: Vector[IndexedParsedForm]): Vector[IndexedParsedForm] = {
    val noIndices: Vector[ParsedForm] = idff.map(_.pform)
    noIndices.zipWithIndex.map( ni => {
        val idx: Int = ni._2
        val pf: ParsedForm = ni._1
        IndexedParsedForm(idx, pf)
    })
}

defined function reindexParsedForms

In [11]:
def pickACard( 
    cards: Vector[IndexedParsedForm], 
    howMany: Int, 
    joker: Option[IndexedParsedForm] ): Vector[IndexedParsedForm] = {

    /*
        After the code below this new `def` is executed, this function
        will be invoked until we've picked the required number of "cards".
    */
    def pickRecurse( 
        ourHand: Vector[IndexedParsedForm], 
        whatsLeft: Vector[IndexedParsedForm]
    ): Vector[IndexedParsedForm] = {
        
        if (ourHand.size == howMany) reindexParsedForms(ourHand) // > because we add the correct answer to the count
        else {
            val limit = whatsLeft.size
            // Get a random next item!
            val r = scala.util.Random
            val pickedIndex: Int = r.nextInt(limit)
            // We have confidence in our re-indexing!
            val pickedCard: IndexedParsedForm = whatsLeft.filter( wl => {
                wl.index == pickedIndex
            }).head
            
            // Add picked card to our "hand"
            val newHand: Vector[IndexedParsedForm] = ourHand :+ pickedCard
            
            // We remove the picked card from the deck
            val newDeck: Vector[IndexedParsedForm] = {
                val withCardRemoved = whatsLeft.filter( _ != pickedCard )
                // And so as not to bore the user with repeated parsings…
                val withCardsFormsRemoved = withCardRemoved.filter( c => {
                    c.pform.parse != pickedCard.pform.parse
                })
                // and re-index…
                reindexParsedForms(withCardsFormsRemoved)
            }
            // recurse!
            pickRecurse( newHand, newDeck)
        }
        
    }
    
    // remove the joker from the deck
    val noJokers: Vector[IndexedParsedForm] = {
        joker match {
        case Some(j) => {
            // We remove the initial joker…
            val origJokerGone: Vector[IndexedParsedForm] = cards.filter( c => {
                c.index != j.index
            })
            
            val dupParsesGone: Vector[IndexedParsedForm] = origJokerGone.filter( c => {
                c.pform.parse != j.pform.parse
            })
            
            // We reindex!
            reindexParsedForms(dupParsesGone)
        }
        case None => cards
        }
    }
    
    /* Now that we've gotten rid of the "joker", we can set up the recurse… */
    
    // emptyVec will be the starting value for `ourHand`
    val emptyVec: Vector[IndexedParsedForm] = Vector[IndexedParsedForm]()
	pickRecurse( emptyVec, noJokers )
}

defined function pickACard

## Make Question

This function assembles everything needed to create a drill question. 

What is needed is:

- A correct answer: a `ParsedForm`
- A Vector of other answers: a `Vector[ParsedForm]`

But it is more complicated than that. Obviously, our correct answer is correct. But it may be that some of our other answers are also correct. E.g. "The feminine genitive singular of 'who?'" might be **τίνος**, but it might be **τοῦ**. If **τοῦ** shows up, randomly, as an "other" answer, we need, not only to give the student credit for picking it, to *insist* that the user pick both **τοῦ** and **τίνος**.

This is where `GradedForm` comes in.

In [12]:
def makeQ( correct: IndexedParsedForm, deck: Vector[IndexedParsedForm], choices: Int = 5): Vector[GradedForm] = {
   
    val otherAnswers: Vector[IndexedParsedForm] = pickACard( deck, choices, Some(correct))
    
    // We might end up with more than one correct answer…
    val numberCorrect: Int = {
        otherAnswers.filter( a => {
            a.pform.form == correct.pform.form
        }).size + 1
    }
    
    // Make GradedForms
    val allAnswers: Vector[IndexedParsedForm] = correct +: otherAnswers
    val gradedAnswers: Vector[GradedForm] = allAnswers.map( aa => {
        val parse = aa.pform.parse
        // Is this one correct?
        val isCorrect: Boolean = ( aa.pform.form == correct.pform.form ) 
        val grade: Int = {
            if (isCorrect) 100 / numberCorrect
            else -100
        }
        GradedForm(grade, parse)
    })
    gradedAnswers
}

defined function makeQ

Turn question-data into a GIFT string.

In [13]:
def makeGiftQuestion(questionIntro: String, prompt: String, answers: Vector[GradedForm], index: Int): String = {
    
    val firstBit: String = s"""::P${index}::[markdown]Identify this form of ${questionIntro}: __${prompt}__ {"""
    val lastBit: String = "}"
    
    val answerStrings: Vector[String] = answers.map( a => {
        s"~%${a.grade}%${a.form}"
    })
    
    val shuffledAnswers: Vector[String] = shuffle(answerStrings)
    
    
    firstBit + "\n" + shuffledAnswers.mkString("\n") + "\n" + lastBit
}

defined function makeGiftQuestion

Make the actual drill!

In [14]:
def shuffleParsedForms( forms: Vector[ParsedForm]): Vector[ParsedForm] = {
    val rr: Vector[Float] = forms.map( i => {
        scala.util.Random.nextFloat
    })
    val zipped: Vector[(ParsedForm,Float)] = forms.zip(rr)
    zipped.sortBy( m => m._2).map(_._1)
}

def makeGiftDrill( 
    dataPath: String, 
    morph: String,
    outputPath: String,
    outputName: String,
    howMany: Int,
    howManyChoices: Int = 5,
    category: Option[String] = None
): Unit = {
    
    // filter out blanks and comments
    val formsLines: Vector[String] = {
        loadFile(s"${dataPath}${morph}").filter(_.size > 0).filter(_.startsWith("""//""") == false)
    }
    
    // extract template page from formsLines
    val templatePath: String = formsLines.head
    
    val forms: Vector[String] = formsLines.tail
    
    // filter out blanks and comments    
    val parsings: Vector[String] = {
        loadFile(s"${dataPath}${templatePath}").filter(_.size > 0).filter(_.startsWith("""//""") == false)
    }
    
    
    val formName: String = forms.head // The first line of a morphology file, used to generate the question.

    val parsedForms: Vector[ParsedForm] = parsings.zip(forms.tail).map( t => {
        ParsedForm( t._1, t._2)
    }).filter( p => {
        p.form != "-" // allows for missing forms in data
    })

    /* 
        Here is where howMany comes in. There are several cases to consider
        - howMany is 0 (= make one for each form in the data)
        - howMany is less than howManyChoices (won't do…)
        - howMany is less than the total number of forms
        - howMany is greater than the total number of forms
    */
    
    val useThese: Vector[ParsedForm] = {
        if (howMany == 0) shuffleParsedForms(parsedForms)
        else {
            if (howMany <= parsedForms.size) {
                if (howMany < howManyChoices) shuffleParsedForms(parsedForms).take(howManyChoices)
                else shuffleParsedForms(parsedForms).take(howMany)
            } else {
                val howManyCopies: Int = (howMany / parsedForms.size) + 1
                val longList: Vector[ParsedForm] = {
                    (1 to howManyCopies).toVector.map(c => {
                        parsedForms
                    }).flatten
                }
                shuffleParsedForms(longList).take(howMany)
            }
        }
    }
    
    val indexedParsedForms: Vector[IndexedParsedForm] = {
        useThese.zipWithIndex.map( t => {
            val pform = t._1
            val idx = t._2
            IndexedParsedForm(idx, pform)
        })
    }
    
    val questions: Vector[String] = {
       indexedParsedForms.map( pf => {
           val prompt: String = pf.pform.form
           val qs: Vector[GradedForm] = {
               makeQ( pf, indexedParsedForms, howManyChoices)
           }
           makeGiftQuestion( formName, prompt, qs, pf.index)
       })
    
    }
    
   val quiz: String = {
        // Includes a hack for how Moodle requires us to do partial credit
        val qStrings: String = questions.mkString("\n\n").replaceAll("%33%","%33.333%")
        val catString: String = category match {
            case Some(s) => "$CATEGORY: " + s
            case None => ""
        }
        catString + "\n\n" + qStrings
    }
    
    //println(quiz)
    
    saveString( quiz, outputPath, s"passive_${outputName}" )

    
}

defined function shuffleParsedForms
defined function makeGiftDrill

## Quiz Configuration

Set up your data:

In [15]:
val morphDataDir: String = "morphology/"
val giftDir: String = "gifts/"

case class QuizChoice(morphData: String, fileName: String, category: String = "" )

val choices: Map[Int, QuizChoice] = Map(
    // 1 = definite article
    1 -> QuizChoice( "adjectives_1/def_article.txt",
            "def_article.gift", "Greek/Morphology/definite_article"),
    // 2 = καλός -ή -όν
    2 -> QuizChoice( "adjectives_1/adjective_kalos.txt",
            "adj_kalos.gift", "Greek/Morphology/kalos"),
    // 3 = participles of εἰμί
    3 -> QuizChoice( "participles_1/participle_eimi.txt",
            "part_eimi.gift"),
    // 4 = πέμπω present and imp. act. indic and pres. act inf.
    4 -> QuizChoice( "verbs_1/verb_pempo_1.txt",
            "verb_pempo_1.gift", "Greek/Morphology/pempo_pres_imp_act"),
    // 5 = πέμπω present, imp, and future active indicative, present and future infinitives
    5 -> QuizChoice( "verbs_1/verb_pempo_2.txt",
            "verb_pempo_2.gift", "Greek/Morphology/pempo_pres_imp_fut_act"),
    // 6 = πέμπω present and imp. act./mid./pass. indic and pres. act., mid., passive inf.
    6 -> QuizChoice( "verbs_1/verb_pempo_1amp.txt",
            "verb_pempo_1amp.gift", "Greek/Morphology/pempo_pres_imp_fut_act_amp"),
    7 -> QuizChoice( "nouns/δοῦλος_1.txt",
            "noun_δοῦλος_1.gift", "Greek/Morphology/nouns12"),
    8 -> QuizChoice( "nouns/ὅπλον.txt",
            "noun_ὅπλον.gift", "Greek/Morphology/nouns12"),
    9 -> QuizChoice( "nouns/εὐχή.txt",
            "noun_εὐχή.gift", "Greek/Morphology/nouns12"),
    10 -> QuizChoice( "nouns/νόσος.txt",
            "noun_νόσος.gift", "Greek/Morphology/nouns12"),
    11 -> QuizChoice( "nouns/νῆσος.txt",
            "noun_νῆσος.gift", "Greek/Morphology/nouns12"),
    12 -> QuizChoice( "nouns/λόγος.txt",
            "noun_λόγος.gift", "Greek/Morphology/nouns12"),
    13 -> QuizChoice( "nouns/θόρυβος.txt",
            "noun_θόρυβος.gift", "Greek/Morphology/nouns12"),
    14 -> QuizChoice( "nouns/θυσία.txt",
            "noun_θυσία.gift", "Greek/Morphology/nouns12"),
    15 -> QuizChoice( "adjectives_1/autos.txt",
            "pronouns_αὐτος.gift", "Greek/Morphology/demonstratives"),
    16 -> QuizChoice( "adjectives_1/hode.txt",
            "pronouns_ὅδε.gift", "Greek/Morphology/demonstratives"),
    17 -> QuizChoice( "adjectives_1/outos.txt",
            "pronouns_οὗτος.gift", "Greek/Morphology/demonstratives"),
    18 -> QuizChoice( "adjectives_1/ekeinos.txt",
            "pronouns_ἐκεῖνος.gift", "Greek/Morphology/demonstratives"),
    19 -> QuizChoice( "verbs_1/verb_pauw_123_ind_inf.txt", "verbs_παύω_123_ind_amp.txt", "Greek/Morphology/06_04_2020"),
    20 -> QuizChoice( "verbs_1/verb_pempo_123amp.txt", "verbs_πέμπω_123_ind_amp.txt", "Greek/Morphology/06_04_2020")

  
    )

val howMany: Int = 50 // How many questions to make, set to 0 to make one item for each form in the data.
val howManyChoices: Int = 4 // How many multiple-choices (in addition to the correct one) to offer

def writeQuiz( quiz: Int, 
              hm: Int = howMany, 
              hmc: Int = howManyChoices,
              mdd: String = morphDataDir, 
              gDir: String = giftDir ) = {
    val qc: QuizChoice = choices(quiz)
    val morphData: String = qc.morphData
    val fileName: String = qc.fileName
    val category: Option[String] = if (qc.category.size > 0) Some(qc.category) else None
    makeGiftDrill( 
        mdd,
        morphData,
        gDir,
        fileName,
        hm,
        hmc,
        category
    )
}





morphDataDir: String = "morphology/"
giftDir: String = "gifts/"
defined class QuizChoice
choices: Map[Int, QuizChoice] = Map(
  5 -> QuizChoice(
    "verbs_1/verb_pempo_2.txt",
    "verb_pempo_2.gift",
    "Greek/Morphology/pempo_pres_imp_fut_act"
  ),
  10 -> QuizChoice(
    "nouns/\u03bd\u03cc\u03c3\u03bf\u03c2.txt",
    "noun_\u03bd\u03cc\u03c3\u03bf\u03c2.gift",
    "Greek/Morphology/nouns12"
  ),
  14 -> QuizChoice(
    "nouns/\u03b8\u03c5\u03c3\u03af\u03b1.txt",
    "noun_\u03b8\u03c5\u03c3\u03af\u03b1.gift",
    "Greek/Morphology/nouns12"
  ),
  20 -> QuizChoice(
    "verbs_1/verb_pempo_123amp.txt",
    "verbs_\u03c0\u03ad\u03bc\u03c0\u03c9_123_ind_amp.txt",
    "Greek/Morphology/06_04_2020"
  ),
  1 -> QuizChoice(
    "adjectives_1/def_article.txt",
    "def_article.gift",
    "Greek/Morphology/definite_article"
  ),
  6 -> QuizChoice(
    "verbs_1/verb_pempo_1amp.txt",
    "verb_pempo_1amp.gift",
    "Greek/Morphology/pempo_pres_imp_fut_act_amp"
  ),
  9 -> QuizChoice(
    "no

## Do It!

In [16]:
// Write a single question-set
//writeQuiz(5)

// Write a single set with 20 questions
writeQuiz(19,20)
writeQuiz(20,20)



// Write all quizzes, with the default number of questions
//for (k <- choices.keys) writeQuiz(k)